In [ ]:
import librosa
import librosa.display
import IPython.display as ipd
import os
import numpy as np

In [ ]:
def feature_extraction(audio_path):
    x, sample_rate = librosa.load(audio_path, res_type="kaiser_fast")
    mfcc = np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=100).T, axis=0)
    return mfcc

In [ ]:
features = {}
i = 0
directory = "E:\\ML project\\codes\\data\\LJSpeech-1.1\\wavs\\"
for audio in os.listdir(directory):
    audio_path = directory + audio
    features[i] = feature_extraction(audio_path)
    if i % 100 == 0:
        print(i)
    i += 1

In [ ]:
features[0]

In [ ]:
# Function to create a 10x10 matrix from an array
def create_10x10_matrix(array):
    if len(array) >= 100:
        return array[:100].reshape(10, 10)
    else:
        return None  # Not enough elements to create a 10x10 matrix


# Create a list to store the matrices
matrix_list = []

# Iterate through the keys and create matrices
for key in features:
    matrix = create_10x10_matrix(features[key])
    if matrix is not None:
        matrix_list.append(matrix)

# Convert the list of matrices to a NumPy array
matrix_array = np.array(matrix_list)

# Example: Print the shape of the resulting array
print("Shape of the matrix array:", matrix_array.shape)
matrix_array[0]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Assuming you have a 3D NumPy array 'matrix_array' containing your 10x10 matrices
# The shape of matrix_array should be (num_matrices, 10, 10)

# Reshape to add the channel dimension
matrix_array_reshaped = matrix_array.reshape(matrix_array.shape[0], 10, 10, 1)
y = np.load("E:\\ML project\\codes\\nlp\\cnn_output.npy")  # normailized
y = y.reshape(-1, 30)

In [ ]:
# Build a simple CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation=None, input_shape=(10, 10, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation=None))
model.add(Dense(30, activation=None))

# Compile the model
model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

In [ ]:
# Train the model
model.fit(matrix_array_reshaped, y, epochs=100, batch_size=32)

In [ ]:
matrix_array_reshaped = matrix_array.reshape(matrix_array.shape[0], 10, 10, 1)

# Make predictions
predictions = model.predict(matrix_array_reshaped)

predictions[0], y[0]

In [ ]:
from sklearn.metrics import mean_squared_error

# Make predictions
predictions = model.predict(matrix_array_reshaped)

# Calculate MSE
mse = mean_squared_error(y, predictions)

print("Mean Squared Error:", mse)